# Assignment #4 (demo). Exploring OLS, Lasso and Random Forest in a regression task
## author: Yury Kashnitsky. All content is distributed under the Creative Commons CC BY-NC-SA 4.0 license.

In [4]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

In [5]:
DATA_PATH = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/main/data/"
data = pd.read_csv(DATA_PATH + "winequality-white.csv", sep=";")

In [6]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


## Separate the target feature, split data in 7:3 proportion (30% form a holdout set, use random_state=17), and preprocess data with StandardScaler.

In [8]:
y = data['quality']
X = data.drop(columns=[ 'quality'])

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y,test_size=0.3,random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

# Linear regression
## Train a simple linear regression model (Ordinary Least Squares).

In [9]:
linreg = LinearRegression().fit(X_train_scaled, y_train)
linreg

LinearRegression()

# Model evaluation
## We evaluate the model’s performance on the test set using the mean squared error and the coefficient of determination.

In [10]:
y_pred = linreg.predict(X_holdout_scaled)

In [11]:
y_pred

array([5.57815637, 6.13413553, 5.93310004, ..., 6.93033664, 5.50467546,
       5.77599116])

In [12]:
print(f"Mean squared error: {mean_squared_error(y_pred, y_holdout):.2f}")

Mean squared error: 0.58


## Sort features by their influence on the target feature (wine quality). Beware that both large positive and large negative coefficients mean large influence on target. It’s handy to use pandas.DataFrame here.

Question 2: Which feature this linear regression model treats as the most influential on wine quality?

In [25]:
coef = linreg.coef_
linreg_coef = pd.DataFrame(coef)
linreg_coef
linreg_coef.sort_values(by=[0], ascending=False)

,0
3,0.538164
8,0.150036
10,0.129533
0,0.097822
9,0.062053
5,0.042180
6,0.014304
4,0.008127
2,-0.000183
1,-0.192260


In [30]:
coef_abs = np.abs(linreg.coef_)
linreg_abs_coef = pd.DataFrame(coef_abs)
linreg_abs_coef
linreg_abs_coef.sort_values(by=[0], ascending=False)

,0
7,0.665720
3,0.538164
1,0.192260
8,0.150036
10,0.129533
0,0.097822
9,0.062053
5,0.042180
6,0.014304
4,0.008127


In [ ]:
Feature 3 → Residual sugar: has the strongest positive impact.
Feature 7 → Density: has the strongest negative impact.

But if we look at the absolute strength, Density actually has the biggest overall impact.

# Lasso regression

In [35]:
from sklearn.linear_model import Lasso
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

In [36]:
#Which feature is the least informative in predicting wine quality, according to this LASSO model?

coef = lasso1.coef_
lasso1_coef = pd.DataFrame(coef)
lasso1_coef
lasso1_coef.sort_values(by=[0], ascending=False)

Object `model` not found.


,0
10,0.322425
3,0.256363
8,0.067277
5,0.043088
9,0.029722
0,-0.000000
2,-0.000000
6,-0.000000
4,-0.002747
1,-0.188479


feature 10 --> alcohol  

In [41]:
#Train LassoCV with random_state=17 to choose the best value of alpha in 5-fold cross-validation.

alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(cv=5, random_state=17, alphas=alphas).fit(X_train_scaled, y_train)
lasso_cv.alpha_



0.0002833096101839324

Question 3: Which feature is the least informative in predicting wine quality, according to the tuned LASSO model?

In [43]:
coef = lasso_cv.coef_
lasso_cv_coef = pd.DataFrame(coef)
lasso_cv_coef
lasso_cv_coef.sort_values(by=[0], ascending=True)

,0
7,-0.648161
1,-0.192049
2,-0.000000
4,0.006933
6,0.012969
5,0.042698
9,0.060939
0,0.093295
10,0.137115
8,0.146549


features 7 --> density 

## Question 4: What are mean squared errors of tuned LASSO predictions on train and holdout sets?

In [ ]:
y_pred_test = lasso_cv.predict(X_holdout_scaled)
print(f"Mean squared error: {mean_squared_error(y_pred, y_holdout):.2f}")

Mean squared error: 0.58


In [47]:
y_pred_train = lasso_cv.predict(X_train_scaled)
print(f"Mean squared error: {mean_squared_error(y_pred_train, y_train):.2f}")

Mean squared error: 0.56


# Random Forest